### Face Recognition and Launching AWS instance

#### Collecting Data

In [ ]:
import cv2
import numpy as np
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)
user_id = input('Enter user ID : ')
cap = cv2.VideoCapture(0)
count = 0
while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=10)
    
    for (x,y,w,h) in faces:
        roi_gray = gray[y:y+h,x:x+w]
        roi_color = frame[y:y+h,x:x+w]
        cv2.imwrite("Hello.png",roi_color)
        color= (255,0,0)
        width = x + w
        height = y + h
        stroke = 2
        count += 1
        cv2.rectangle(frame,(x,y),(width,height),color,stroke)
        file_name_path = './faces/User_' + str(user_id) + "_" + str(count) + '.jpg'
        cv2.imwrite(file_name_path, roi_gray)
        cv2.putText(frame, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
      
    if cv2.waitKey(1) == 13 or count == 100: 
        break

    cv2.imshow('frame',frame)
cap.release()
cv2.destroyAllWindows()
print("100 Images Capatured for training.")

#### Training Model

In [ ]:
import cv2
import numpy as np
import os
from os import listdir
from os.path import isfile, join

data_path = './faces/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

Training_Data, Labels = [], []

for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    user_id=int(os.path.split(image_path)[-1].split("_")[1])
    Labels.append(user_id)

Labels = np.asarray(Labels, dtype=np.int32)
model  = cv2.face_LBPHFaceRecognizer.create()
model.train(np.asarray(Training_Data), np.asarray(Labels))
model.save('./trained-model.rh')
print("success")

#### Performing Recognization

In [ ]:
import cv2
import os
import mail

cap = cv2.VideoCapture(0)
count = 0
count1 = 0

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trained-model.rh')

def face_detector(img, size=0.5):
    face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.2, 10)
    if faces == ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi

while True:

    ret, frame = cap.read()
    
    image, face = face_detector(frame)
    
    try:
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

            results = recognizer.predict(face)
           
            if results[1] < 50:
                confidence = int( 100 * (1 - (results[1])/400) )
                display_string = str(confidence) + '% Confident it is User'
            

            if confidence > 80 and count <=20 and results[1] < 50 :

                count  = count + 1
            
                cv2.putText(image, "Welcome Back User!!", (250, 100), cv2.FONT_HERSHEY_COMPLEX, 1, (120,21,207), 2)
                cv2.imshow('Main', image )
                if count == 15 :
                    
                    mail.recognize()
                    os.system('start chrome "https://api.callmebot.com/whatsapp.php?phone=+9191000000000&text=Welcome Back Sushrut. Launching workspace. &apikey=*****" -incognito')
                    print("Whatsapp Message send")
                    os.system('terraform apply -auto-approve')
                    print("Workspace Launch")
                    break


            else: 
                count1 = count1 + 1
                cv2.putText(image, "Face not Recognized!!", (250, 100), cv2.FONT_HERSHEY_COMPLEX, 1, (120,21,207), 2)
                cv2.imshow('Main', image )
                cv2.imwrite("unknown-person.jpg",image)
                if count1 == 10 :
                    os.system('start chrome "https://api.callmebot.com/whatsapp.php?phone=+911000000000&text=Alert, Someone has Checked into system. Hence,terminating workspace. Check mail for details. &apikey=*****" -incognito')
                    print("Whatsapp Message send")
                    mail.unrecognize()
                    os.system('terraform destroy -auto-approve')
                    print("Workspace Terminated")
                    break
            
    except Exception as e:
            cv2.imshow('Main', image )
            pass

     
    if cv2.waitKey(10) == 13: #13 is the Enter Key
        break

cap.release()
cv2.destroyAllWindows()     #(209, 80, 0, 255